In [1]:
#importing libraries 
import pandas as pd 
import matplotlib as plt 
import numpy as np 
import logging
import json
import warnings
import ast

In [2]:
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [3]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)

In [4]:
df = pd.read_csv("/kaggle/input/tmdb-raw/movies_raw.csv")
#pd.set_option('display.max_colwidth',50)
df

,0
0,NaN
1,"{'adult': False, 'backdrop_path': '/9wXPKruA6b..."
2,"{'adult': False, 'backdrop_path': '/7JNzw1tSZZ..."
3,"{'adult': False, 'backdrop_path': '/8BTsTfln4j..."
4,"{'adult': False, 'backdrop_path': '/mDfJG3LC3D..."
5,"{'adult': False, 'backdrop_path': '/xnHVX37XZE..."
6,"{'adult': False, 'backdrop_path': '/dF6FjTZzRT..."
7,"{'adult': False, 'backdrop_path': '/1TUg5pO1VZ..."
8,"{'adult': False, 'backdrop_path': '/9BBTo63ANS..."
9,"{'adult': False, 'backdrop_path': '/ehzI1mVcnH..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       18 non-null     object
dtypes: object(1)
memory usage: 284.0+ bytes


> # Master Function


In [6]:
def clean_movies_data(df: pd.DataFrame, validate: bool=True, log_path: str=None) -> pd.DataFrame:
    """
    Step 2: Data Cleaning and Preprocessing for TMDB Movie Data
    
    This function performs:
    - Conversion of nested JSON-like strings to proper Python objects
    - Dropping irrelevant columns
    - Extraction of key information from JSON-like columns
    - Handling missing and incorrect data
    - Replacement of unrealistic values
    - Removing duplicates and filtering
    - Column reordering and placeholders creation
    - Resetting index and preparing the final clean dataset
    
    Args:
        df (pd.DataFrame): Raw TMDB movie dataframe
        validate (bool): If True, logs column anomaly info
        log_path (str): Optional path to save log file
    
    Returns:
        pd.DataFrame: Cleaned movie dataframe ready for analysis
    """
    import logging

    # Create logger
    logger = logging.getLogger("movie_cleaner")
    logger.setLevel(logging.INFO)

    # If writing to log file
    if log_path:
        handler = logging.FileHandler(log_path)
    else:
        handler = logging.StreamHandler()   # logs to console if no file provided

    # Create log format
    formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
    handler.setFormatter(formatter)

    # Avoid duplicate handlers if function is called multiple times
    if not logger.handlers:
        logger.addHandler(handler)

    logger.info(f"Starting cleaning pipeline. Input shape: {df.shape}")


    logging.info(f"Starting cleaning pipeline. Input shape: {df.shape}")
    # Step 2a: Convert to proper DataFrame
    df = df[df.iloc[:,0].notna()] 
    df_dicts = df.iloc[:,0].apply(ast.literal_eval)
    df = pd.json_normalize(df_dicts)
    
    # Step 2b: Drop irrelevant columns
    cols_to_drop = ['adult', 'imdb_id', 'original_title', 'video', 'homepage']
    df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])
    
    # Step 2c: Convert JSON-like columns to proper objects
    df['belongs_to_collection'] = df['belongs_to_collection'].apply(
        lambda x: x if isinstance(x, dict) else ast.literal_eval(x) if isinstance(x, str) else {}
    )
    
    json_cols = ['genres', 'production_countries', 'production_companies', 'spoken_languages']
    for col in json_cols:
        df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
        df[col] = df[col].apply(lambda x: [] if x is None else x)
    
    # Step 2d: Extract key information
    df['collection_name'] = df['belongs_to_collection'].apply(
        lambda x: x.get('name', '') if isinstance(x, dict) else ''
    )
    df['genres'] = df['genres'].apply(lambda x: '|'.join([g['name'] for g in x]))
    df['spoken_languages'] = df['spoken_languages'].apply(lambda x: '|'.join(d['english_name'] for d in x))
    df['production_countries'] = df['production_countries'].apply(lambda x: '|'.join(c['name'] for c in x))
    df['production_companies'] = df['production_companies'].apply(lambda x: '|'.join(c['name'] for c in x))
    
    # Step 2e: Inspect anomalies (optional)
    if validate:
        for col in ['collection_name', 'genres', 'spoken_languages', 'production_countries', 'production_companies']:
            empty_count = df[col].isin(["", "[]", "{}"]).sum()
            unique_vals = df[col].nunique()
            logger.info(f"Column '{col}': {empty_count} empty/placeholder values, {unique_vals} unique values")
    

    # Step 2f: Convert column datatypes
    df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
    df['id'] = pd.to_numeric(df['id'], errors='coerce')
    df['popularity'] = pd.to_numeric(df['popularity'], errors='coerce')
    df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
    
    numeric_cols = ['revenue', 'vote_average', 'vote_count', 'belongs_to_collection.id']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
    
    # Step 2g: Replace unrealistic values
    df[['budget', 'revenue', 'runtime']] = df[['budget', 'revenue', 'runtime']].replace(0, np.nan)
    df['budget'] = df['budget'] / 1_000_000
    df['revenue'] = df['revenue'] / 1_000_000
    df.loc[df['vote_count'] == 0, 'vote_average'] = np.nan
    
    placeholder_texts = ['No Data', 'N/A', 'Unknown', '']
    df['overview'] = df['overview'].replace(placeholder_texts, np.nan)
    df['tagline'] = df['tagline'].replace(placeholder_texts, np.nan)
    
    # Step 2h: Remove duplicates and filter
    df = df.drop_duplicates(subset=['id', 'title'])
    df = df.dropna(thresh=10)
    if 'status' in df.columns:
        df = df[df['status'] == 'Released'].drop(columns=['status'])
    
    # Step 2i: Rename columns
    df = df.rename(columns={'budget': 'budget_musd', 'revenue': 'revenue_musd'})
    
    # Step 2j: Create empty placeholders for missing columns
    df['cast'] = [[]] * len(df)
    df['cast_size'] = 0
    df['director'] = [[]] * len(df)
    df['crew_size'] = 0
    
    # Step 2k: Reorder columns
    final_cols = [
        'id', 'title', 'tagline', 'release_date', 'genres', 'belongs_to_collection',
        'original_language', 'budget_musd', 'revenue_musd', 'production_companies',
        'production_countries', 'vote_count', 'vote_average', 'popularity', 'runtime',
        'overview', 'spoken_languages', 'poster_path', 'cast', 'cast_size', 'director', 'crew_size'
    ]
    df = df[[c for c in final_cols if c in df.columns]]
    
    # Step 2l: Reset index-
    df.reset_index(drop=True, inplace=True)
    
    logger.info("Movie data cleaned successfully.")
    
    return df

In [7]:
df= pd.read_csv('/kaggle/input/tmdb-raw/movies_raw.csv')
cleaned_df = clean_movies_data(df, validate=True, log_path="cleaning.log")
cleaned_df.to_csv("tmdb_movies_clean.csv", index=False)

2025-12-08 08:56:19,090 [INFO] Starting cleaning pipeline. Input shape: (19, 1)
2025-12-08 08:56:19,091 [INFO] Starting cleaning pipeline. Input shape: (19, 1)
2025-12-08 08:56:19,108 [INFO] Column 'collection_name': 18 empty/placeholder values, 1 unique values
2025-12-08 08:56:19,110 [INFO] Column 'genres': 0 empty/placeholder values, 14 unique values
2025-12-08 08:56:19,111 [INFO] Column 'spoken_languages': 0 empty/placeholder values, 10 unique values
2025-12-08 08:56:19,112 [INFO] Column 'production_countries': 0 empty/placeholder values, 3 unique values
2025-12-08 08:56:19,114 [INFO] Column 'production_companies': 0 empty/placeholder values, 13 unique values
2025-12-08 08:56:19,133 [INFO] Movie data cleaned successfully.
